In [ ]:
import random
from pathlib import Path

import graphix.command
import numpy as np
import stim
import veriphix.client
from graphix.fundamentals import IXYZ
from graphix.noise_models import DepolarisingNoiseModel
from graphix.pauli import Pauli
from graphix.random_objects import Circuit, rand_circuit
from graphix.sim.density_matrix import DensityMatrixBackend
from graphix.sim.statevec import Statevec, StatevectorBackend
from graphix.states import BasicStates
from veriphix.client import CircuitUtils, Client, Secrets


import gospel.brickwork_state_transpiler
from gospel.scripts.qasm2brickwork_state import read_qasm, draw_brickwork_state_colormap

In [ ]:
## Load a circuit with success probability p = 0.7839549798834848
# BQP error
# context handler open renvoie f et à la fin ferme le fichier
# valeur à durer de vie, resource libéré.

with Path("circuits/circuit0000.qasm").open() as f:
    circuit = read_qasm(f)

print(circuit.instruction)

pattern = gospel.brickwork_state_transpiler.transpile(circuit)

print(list(pattern))


## Measure output nodes, to have classical output
classical_output = pattern.output_nodes
for onode in classical_output:
    pattern.add(graphix.command.M(node=onode))

states = [BasicStates.PLUS] * len(pattern.input_nodes)

# correct since the pattern is transpiled from a circuit and hence has a causal flow
pattern.minimize_space()

print(f"Number of nodes in the pattern : {pattern.n_node}")

### Here comes **Veriphix**

#### Test rounds
Setting up the client, the simulation, tests rounds, plotting

In [ ]:
secrets = Secrets(r=True, a=True, theta=True)
client = Client(pattern=pattern, secrets=secrets)

# define test runs using a greedy colouring algorithm
# test_runs =  client.create_test_runs()
# for i in test_runs:
#     print(i.traps_list)

# checking new feature of veriphix
# don't use heuristic color-finding but use the optimal one (bipartite)
colours = gospel.brickwork_state_transpiler.get_bipartite_coloring(pattern) 
test_runs =  client.create_test_runs(manual_colouring=colours)

# for j, i in enumerate(test_runs):
#     print(j)
#     print(i.traps_list)

# Insert noise here 
noise = DepolarisingNoiseModel(entanglement_error_prob=0.1)

backend = DensityMatrixBackend()

n_failures = 0
n_iterations = 10 

# print(test_runs[0].traps_list)  # test_runs

# list n_iteration : list of traps (nodes) that returned one
test_outcome_table = []

for _ in range(n_iterations):
    run = random.choice(test_runs)
    # print(run.traps_list)
    trap_outcomes = client.delegate_test_run(
        run=run, backend=backend, noise_model=noise
    )
    # print(trap_outcomes)
    result = {
        trap: outcome for (trap,), outcome in zip(run.traps_list, trap_outcomes)
    }  # extrait un elet de la structure virgule nécessair epour vérifier un seul élément, ou next(iter(trap))
    # both are in same order
    test_outcome_table.append(result)

    if sum(trap_outcomes) != 0:
        n_failures += 1
        print("Failed trap round")
    else:
        print("Trap round passed")

print(f"Number of failed rounds: {n_failures}/{n_iterations}")

### Plotting the trap failure data
Working on the output first...

In [ ]:
print(len(test_outcome_table))
occurences = {}
occurences_one = {}

for results in test_outcome_table:
    for q, r in results.items():
        if q not in occurences:
            occurences[q] = 1
            occurences_one[q] = r
        else:
            occurences[q] += 1
            if r == 1:
                occurences_one[q] += 1

failure_proba = {q: occurences_one[q] / occurences[q] for q in occurences}
print(failure_proba)


In [ ]:
# change this to save the figure
path = Path("simulation/")
target = "pic2.svg"

draw_brickwork_state_colormap(circuit=circuit, target=path / target, failure_probas=failure_proba)

#### Full computation (computation and tests rounds)

todo

In [ ]:
def make_clifford(pattern: graphix.Pattern):
    clifford_pattern = graphix.Pattern(pattern.input_nodes)
    for cmd in pattern:
        clifford_pattern.add(cmd)
    for cmd in clifford_pattern:
        if cmd.kind == graphix.command.CommandKind.M:
            new_angle = random.choice([0, 1 / 2, 1, 3 / 2])
            cmd.angle = new_angle
    return clifford_pattern

In [ ]:
backend = DensityMatrixBackend()


# Noiseless computation run
client.delegate_pattern(backend=backend)
for onode in classical_output:
    print(client.results[onode])


# Noiseless trap run
noiseless_model = DepolarisingNoiseModel(entanglement_error_prob=0)
rd_run = random.choice(runs)
trap_outcomes = client.delegate_test_run(
    backend=backend, run=rd_run, noise_model=noiseless_model
)
print(
    f"Trap outcome of noiseless trap run: {sum(trap_outcomes)}"
)  # should output 0 (noiseless simulation)

# (ultra) Noisy trap run
ultra_noisy_model = DepolarisingNoiseModel(entanglement_error_prob=1)
rd_run = random.choice(runs)
trap_outcomes = client.delegate_test_run(
    backend=backend, run=rd_run, noise_model=ultra_noisy_model
)
print(
    f"Trap outcome of noisy trap run: {sum(trap_outcomes)}"
)  # should NOT output 0 (noisy simulation)
